# **Reinforcement Learning: Taller 4**
## Estudiantes: Juan Pablo Reyes Fajardo y Santiago Rodríguez Ávila 

In [6]:
import gymnasium as gym
import numpy as np
import itertools
import operator
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

import visualkeras

In [ ]:
env = gym.make('CartPole-v1')

# 1. RL Tabular

In [ ]:
# Funcion Auxiliar para diccionarios
def key_max(d):
        return max(d.items(), key=operator.itemgetter(1))
def key_min(d):
        return min(d.items(), key=operator.itemgetter(1))

Política $\epsilon$ - Greedy 

(Útil más adelante)

In [ ]:
def eps_greedy(Q_, state,epsilon=0.1):
    rand = np.random.uniform()
    if rand>epsilon:
        return key_max(Q_[state])[0],1-epsilon
    else:
        return key_min(Q_[state])[0],epsilon

## Discretización

Inicialmente se realizan múltiples experimentos para determinar límites razonables para las variables a discretizar (aquellas cuyo espacio de observación es infinito)

In [ ]:
velocidades_absolutas_maximas={"Lineal":[],"Angular":[]}

observation, info = env.reset()

abs_lineal=abs(observation[1])
abs_angular=abs(observation[3])

for _ in range(int(1e6)):
    
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)
    
    if abs(observation[1])>abs_lineal:
        abs_lineal=abs(observation[1])
    if abs(observation[3])>abs_angular:
        abs_angular=abs(observation[3])

    if terminated or truncated:
        observation, info = env.reset()
        velocidades_absolutas_maximas["Lineal"].append(abs_lineal)
        velocidades_absolutas_maximas["Angular"].append(abs_angular)

env.close()

vel_lin_abs_max=np.mean(velocidades_absolutas_maximas["Lineal"])
vel_ang_abs_max=np.mean(velocidades_absolutas_maximas["Angular"])
print(f'Promedio de velocidad lineal absoluta máxima: \
      {vel_lin_abs_max} \
      \nPromedio de velocidad angular absoluta máxima: \
      {vel_ang_abs_max}')

Discretización de estados:

In [ ]:
# Límites del espacio de observación del MDP real
cart_high_var = env.observation_space.high
cart_low_var = env.observation_space.low

# Espacios de observación discretizados
observation_space_discrete_400=[np.linspace(cart_low_var[0], cart_high_var[0], num= 5),\
                            np.linspace(-vel_lin_abs_max, vel_lin_abs_max, num= 4),\
                            np.linspace(cart_low_var[2], cart_high_var[2], num = 5),\
                            np.linspace(-vel_ang_abs_max, vel_ang_abs_max, num= 4)]\

observation_space_discrete_4096=[np.linspace(cart_low_var[0], cart_high_var[0], num= 8),\
                            np.linspace(-vel_lin_abs_max, vel_lin_abs_max, num= 8),\
                            np.linspace(cart_low_var[2], cart_high_var[2], num = 8),\
                            np.linspace(-vel_ang_abs_max, vel_ang_abs_max, num= 8)]

# Uso de iteradores para obtener todos los estados a partir del espacio de obsevación discreto
states_400=list(itertools.product(*observation_space_discrete_400))
states_4096=list(itertools.product(*observation_space_discrete_4096))

def init_Q_400():
    Q_table={}
    for i in states_400:
        Q_table[i] = {0:0,1:0}
    return Q_table

def init_Q_4096():
    Q_table={}
    for i in states_4096:
        Q_table[i] = {0:0,1:0}
    return Q_table
         
def discretize_400(new_state):
    # car pos, car vel, pole angle, pole vel
    discretizacion=[0]*4
    for i in range(len(discretizacion)):
        dif = [(abs(x - new_state[i])) for x in observation_space_discrete_400[i]]
        discretizacion[i] = observation_space_discrete_400[i][dif.index(min(dif))]
        
    return tuple(discretizacion)

def discretize_4096(new_state):
    # car pos, car vel, pole angle, pole vel
    discretizacion=[0]*4
    for i in range(len(discretizacion)):
        dif = [(abs(x - new_state[i])) for x in observation_space_discrete_4096[i]]
        discretizacion[i] = observation_space_discrete_4096[i][dif.index(min(dif))]
        
    return tuple(discretizacion)

## Estimación de Q con Q-Learning 

### 400 Estados

In [ ]:
def Q_Learning_400(gamma,alpha):
    terminated=False
    observation, info = env.reset()
    observation=discretize_400(observation)
    while not terminated:
        
        action,_=eps_greedy(Q,observation,0.1)
        
        observation_, reward, terminated, truncated, info = env.step(action)
        observation_=discretize_400(observation_)
        
        Q_=key_max(Q[observation_])[1]

        Q[observation][action]+=alpha*(reward+gamma*Q_-Q[observation][action])
        observation=observation_

In [ ]:
env = gym.make('CartPole-v1',render_mode='human')
Q=init_Q_400()
policy=dict.fromkeys(states_400, 0)
for i in range(0,10):
    Q_Learning_400(0.9,0.1)

In [ ]:
def Q_Learning_4096(gamma,alpha):
    terminated=False
    observation, info = env.reset()
    observation=discretize_4096(observation)
    while not terminated:
        
        action,_=eps_greedy(Q,observation,0.1)
        
        observation_, reward, terminated, truncated, info = env.step(action)
        observation_=discretize_4096(observation_)
        
        Q_=key_max(Q[observation_])[1]

        Q[observation][action]+=alpha*(reward+gamma*Q_-Q[observation][action])
        observation=observation_

In [ ]:
env = gym.make('CartPole-v1')
Q=init_Q_4096()
policy=dict.fromkeys(states_4096, 0)
    
for i in tqdm(range(int(1e6))):
    Q_Learning_4096(0.9,0.1)

In [ ]:
for state_4096 in states_4096:
    policy[state_4096],_=eps_greedy(Q,tuple(state_4096),0)

In [ ]:

env = gym.make('CartPole-v1',render_mode="human")


rwds=[]
for _ in range(100):
    observation, info = env.reset()
    r=0
    while True:
        action = policy[discretize_4096(observation)]
        observation, reward, terminated, truncated, info = env.step(action)
        r+=reward

        if terminated or truncated:
            observation, info = env.reset()
            rwds.append(r)
            break
env.close()
print(np.mean(rwds))

In [8]:

ENV_NAME = 'CartPole-v1'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)

nb_actions = env.action_space.n

model = Sequential(
    [
        Flatten(input_shape=(1,) + env.observation_space.shape),
        Dense(16, activation="relu"),
        Dense(2, activation="linear"),
    ]
)


print(model.summary())

policy = EpsGreedyQPolicy()

memory = SequentialMemory(limit=50000, window_length=1)

dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)

dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

dqn.fit(env, nb_steps=100, visualize=True, verbose=2)

visualkeras.layered_view(model)

dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_2 (Dense)             (None, 16)                80        
                                                                 
 dense_3 (Dense)             (None, 2)                 34        
                                                                 
Total params: 114
Trainable params: 114
Non-trainable params: 0
_________________________________________________________________
None
Training for 100 steps ...


ValueError: Error when checking input: expected flatten_1_input to have shape (1, 4) but got array with shape (1, 2)

In [1]:

!python --version
print('NumPy', np.__version__)
print('Tensorflow', tensorflow.__version__)

Python 3.9.13


NameError: name 'np' is not defined

In [5]:
!pip install keras
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


ENV_NAME = 'CartPole-v1'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)

nb_actions = env.action_space.n

# Next, we build a very simple model.
model = Sequential()
model.add(Input(shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
dqn.fit(env, nb_steps=50000, visualize=True, verbose=2)

# After training is done, we save the final weights.
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=True)

ModuleNotFoundError: No module named 'keras'

In [ ]:
#SARSA algorithm
def SARSA_algorithm(num_episodes):
    allowed_actions=[0,1]
    # Variables: alpha, gamma y epsilon.
    alpha = 0.3
    gamma = 1
    epsilon = 0.1
    cont = 0
    #Para 5000 episodios de entrenamiento
    for i in range(num_episodes):
        # Inicializa las variables para cada episodio
        state=env.reset()
        num_steps = 0
        state = discretize(state[0])


        # Seleccion accion "a" de forma epsilon-greedy
        if epsilon< np.random.uniform():        
            act_arg = np.array([Q_table[tuple(state), act] for act in range(2)])
            action = allowed_actions[np.argmax(act_arg)]
        else:
            action = env.action_space.sample()

        # Inicia el episodio
        continue_episode = True
        while continue_episode:        
            # Obtengo s'
            new_state, reward, done, _,_=env.step(action)
            new_state = discretize(new_state)

            # Revisa que new_state no sea un estado terminal
            if done:
                # Valor q(s',a') terminal
                q_value_next_step = 0
            else:
                #Obtengo a' de s' con epsilon greedy
                if epsilon< np.random.uniform():        
                    act_arg = np.array([Q_table[tuple(state), act] for act in range(2)])
                    new_action = allowed_actions[np.argmax(act_arg)]
                else:
                    new_action = env.action_space.sample()

                # Valor q(s',a') no terminal
                q_value_next_step = Q_table[tuple(new_state),new_action]


            # Calculo de actualizacion q(s,a) <- q(s,a) + alpha*(R + gamma*q(s',a') - q(s,a))
            Q_table[tuple(state), action] += alpha*(reward + gamma*q_value_next_step - Q_table[tuple(state),action])

            # asigna a = a' y s = s'
            state = new_state
            action = new_action


            # Parte que termina el episodio si se llega a algun estado terminal
            if done:
                continue_episode = False
        cont+=1
        if cont%100==0:
            print(cont)

## SARSA con 400 estados:

In [ ]:
set_states()
set_q_table()

In [ ]:
discrete_values = num_states(400)
SARSA_algorithm(5000)

In [ ]:
allowed_actions= [0,1]
prom=[]
prom_rand=[]
for i in range(100):
    G_pi = 0
    state =env.reset()
    state = state[0]
    state = discretize(state)
    act_arg = np.array([Q_table[tuple(state), act] for act in range(2)])
    action = allowed_actions[np.argmax(act_arg)]
    done = False
    n=1
    while not done:
        new_state, reward, done, _,_=env.step(action)
        G_pi= G_pi + reward
        discrete_state = discretize(new_state)
        act_arg = np.array([Q_table[tuple(discrete_state), act] for act in range(2)])
        action = allowed_actions[np.argmax(act_arg)]
    prom.append(G_pi)
print('---'*5)
for i in range(100):
    G_pi = 0
    state =env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        new_state, reward, done, _,_=env.step(action)
        G_pi= G_pi + reward
    prom_rand.append(G_pi)

In [ ]:
print("Retorno obtenido con la funcion q:",sum(prom)/len(prom))
print("Retorno obtenido de acciones al azar:",sum(prom_rand)/len(prom_rand))

## SARSA con 4000 estados:

In [ ]:
set_states()
set_q_table()

In [ ]:
discrete_values = num_states(4000)
SARSA_algorithm(1000)

In [ ]:
prom=[]
prom_rand=[]
allowed_actions= [0,1]
for i in range(100):
    G_pi = 0
    state =env.reset()
    state = state[0]
    state = discretize(state)
    act_arg = np.array([Q_table[tuple(state), act] for act in range(2)])
    action = allowed_actions[np.argmax(act_arg)]
    done = False
    n=1
    while not done:
        new_state, reward, done, _,_=env.step(action)
        G_pi= G_pi + reward
        discrete_state = discretize(new_state)
        act_arg = np.array([Q_table[tuple(discrete_state), act] for act in range(2)])
        action = allowed_actions[np.argmax(act_arg)]
    prom.append(G_pi)
print('---'*5)
for i in range(100):
    G_pi = 0
    state =env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        new_state, reward, done, _,_=env.step(action)
        G_pi= G_pi + reward
    prom_rand.append(G_pi)

In [ ]:
print("Retorno obtenido con la funcion q:",sum(prom)/len(prom))
print("Retorno obtenido de acciones al azar:",sum(prom_rand)/len(prom_rand))

In [ ]:
env.close()